# TP06 : Naïve Bayes

Tout le monde connait le théorème de Bayes pour calculer la probabilité conditionnelle d'un évennement $A$ sachant un autre $B$: 
$$ P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$

Pour appliquer ce théorème sur un problème d'appentissage automatique, l'idée est simple ; Etant donné une caractéristique $f$ et la sortie $y$ qui peut avoir la classe $c$ : 
- Remplacer $A$ par $y=c$
- Remplacer $B$ par $f$ 
On aura l'équation : 
$$ P(y=c|f) = \frac{P(y=c)P(f|y=c)}{P(f)}$$

On appelle : 
- $P(y=c|f)$ postérieure 
- $P(y=c)$ antérieure
- $P(f|y=c)$ vraisemblance
- $P(f)$ évidence 

Ici, on estime la probablité d'une classe $c$ sachant une caractéristique $f$ en utilisant des données d'entrainement. Maintenant, on veut estimer la probabilité d'une classe $c$ sachant un vecteur de caractéristiques $\overrightarrow{f} = \{f_1, ..., f_L\}$ : 
$$ P(y=c|\overrightarrow{f}) = \frac{P(y=c)P(\overrightarrow{f}|y=c)}{P(f)}$$

Etant donnée plusieurs classes $c_j$, la classe choisie $\hat{c}$ est celle avec la probabilité maximale 
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k|\overrightarrow{f})$$
$$\hat{c} = \arg\max\limits_{c_k} \frac{P(y=c_k)P(\overrightarrow{f}|y=c_k)}{P(f)}$$
On supprime l'évidence pour cacher le crime : $P(f)$ ne dépend pas de $c_k$ et elle est postive, donc ça ne va pas affecter la fonction $\max$.
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k)P(\overrightarrow{f}|y=c_k)$$

Pour calculer $P(\overrightarrow{f}|y=c_k)$, on va utiliser une properiété naïve (d'où vient le nom Naive Bayes) : on suppose l'indépendence conditionnelle entre les caractéristiques $f_j$. 
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k) \prod\limits_{f_j \in \overrightarrow{f}} P(f_j|y=c_k)$$

Pour éviter la disparition de la probabilité (multiplication et représentation de virgule flottante sur machine), on transforme vers l'espace logarithme.
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$


## Avantages 

Les classifieurs naïfs bayésiens, malgré leur simplicité, ont des points forts:
- Ils ont besoin d'une petite quantité de données d'entrainement.
- Ils sont très rapides par rapport aux autres classifieurs.
- Ils donnent de bons résultats dans le cas de filtrage du courrier indésirable et de classification de documents.

## Limites
Les classifieurs naïfs bayésiens certes sont populaires à cause de leur simplicité. Mais, une telle simplicité vient avec un coût [référence: Spiderman].
- Les probabilités obtenues en utilisant ces classifieurs ne doivent pas être prises au sérieux.
- S'il existe une grande corrélation entre les caractéristiques, ils vont donner une mauvaise performance.
- Dans le cas des caractéristiques continues (prix, surface, etc.), les données doivent suivre la loi normale.


## I- Implémentation

Pour estimer la vraisemblance, il y a plusieurs modèles (lois):
- Loi multinomiale : pour les caracétristiques nominales
- Loi de Bernoulli : lorsqu'on est interressé par l'apparence d'une caractéristique ou non (binaire)
- loi normale : pour les caractéristiques numériques

Dans ce TP, on va implémenter Naive Bayes pour les caractéristiques nominales (loi multinomiale)

### I-1- Les données pour les tests unitaires
Ici, on va utiliser le dataset "jouer" contenant des caractéristiques nominales.

In [1]:
import numpy as np
import pandas as pd 


jouer = pd.read_csv("datasets/jouer.csv")

X_jouer = jouer.iloc[:, :-1].values # Premières colonnes 
Y_jouer = jouer.iloc[:,-1].values # Dernière colonne 

# Afficher le dataset "jouer"
jouer

,temps,temperature,humidite,vent,jouer
0,ensoleile,chaude,haute,non,non
1,ensoleile,chaude,haute,oui,non
2,nuageux,chaude,haute,non,oui
3,pluvieux,douce,haute,non,oui
4,pluvieux,fraiche,normale,non,oui
5,pluvieux,fraiche,normale,oui,non
6,nuageux,fraiche,normale,oui,oui
7,ensoleile,douce,haute,non,non
8,ensoleile,fraiche,normale,non,oui
9,pluvieux,douce,normale,non,oui


### I-2- Estimation de la probabilité antérieure
Etant donné le vecteur de sortie $Y$, on doit calculer la probabilité de chaque classe (différentes valeurs de $Y$)

$$p(c_k) = \frac{|\{y / y \in Y \text{ et } y = c_k\}|}{|Y|}$$

La fonction doit retourner un dictionnaire où la clé est le nom de la classe et la valeur est sa probabilité. Voici, un exemple d'un dictionnaire dans Python

In [2]:
# Exemple de dictionnaire dans Python 
d = {}
d["Iris-setosa"] = 0.5
d["Iris-versicolor"] = 0.33
d["Iris-virginica"] = 0.67

for c in d: 
    print("P(" + c + ")= " + str(d[c]))

P(Iris-setosa)= 0.5
P(Iris-versicolor)= 0.33
P(Iris-virginica)= 0.67


In [11]:
# TODO Réaliser la fonction 
def P_c(Y): 
    keys,counts = np.unique(Y,return_counts=True)
    resultat = {}
    for i in range(len(keys)):
        resultat[keys[i]] = counts[i] / len(Y)
    return resultat

# Résultat: {'non': 0.35714285714285715, 'oui': 0.6428571428571429}
P_c(Y_jouer)

{'non': 0.35714285714285715, 'oui': 0.6428571428571429}

### I-3- Entrainement  (loi multinomiale)

Notre modèle (notons le par $\theta_{f_j,C}$) doit garder le nombre des différentes valeurs dans une caractéristique $A$ et le nombre de ces valeurs dans chaque classe.

Donc, étant donné un vecteur d'une caractéristique $A$ et un autre des $Y$ respectives, la fonction d'entrainement doit retourner un dictionnaire (notre théta) : 
- la clé est une valeur $a_v$ de $A$ 
- la valeur est un autre dictionnaire : 
   - il doit contenir une clé "_total_" dont la valeur est le nombre d'occurence de $a_v$ dans $A$ 
   - la clé est la classe $c_k$ de $Y$
   - la valeur est le nombre d'occurence de $a_v$ respectives à $c_k$


In [24]:
# TODO Réaliser cette fonction 
# Elle génère théta pour une seule caractéristique
def entrainer_multi_1(A, Y): 
    keys,counts = np.unique(A,return_counts=True)
    y_keys,y_counts = np.unique(Y,return_counts=True)
    resultat = {}
    sub_y = sub_y_keys = sub_y_counts = []
    for i in range(len(keys)):
        sub_y = [ val for (index,val) in enumerate(Y) if A[index] == keys[i]]
        sub_y_keys,sub_y_counts = np.unique(sub_y,return_counts=True)
        sub_result = {
            '_total_' : counts[i]
        }
        for k in range(len(y_keys)):
            sub_result[y_keys[k]] = 0
            
        for j in range(len(sub_y_keys)):
            sub_result[sub_y_keys[j]] = sub_y_counts[j] 
        
        resultat[keys[i]] = sub_result
    return resultat

# Résultat 
# {             'ensoleile': {'_total_': 5, 'non': 3, 'oui': 2},
#               'nuageux'  : {'_total_': 4, 'non': 0, 'oui': 4},
#               'pluvieux' : {'_total_': 5, 'non': 2, 'oui': 3}
# }
Theta_jouer_temps = entrainer_multi_1(X_jouer[:, 0], Y_jouer)

Theta_jouer_temps

{'ensoleile': {'_total_': 5, 'non': 3, 'oui': 2},
 'nuageux': {'_total_': 4, 'non': 0, 'oui': 4},
 'pluvieux': {'_total_': 5, 'non': 2, 'oui': 3}}

In [25]:
# La fonction qui entraine Théta sur plusieurs caractéristiques
# Rien à programmer ici
# Notre théta est une liste des dictionnaires;
# chaque dictionnaire contient le théta de la caractéristique respective à la colonne de X
# On ajoute les probabilités antérieures des classes à la fin de résultat
def entrainer_multi(X, Y): 
    resultat = []
    for i in range(X.shape[1]): 
        resultat.append(entrainer_multi_1(X[:, i], Y))
    resultat.append(P_c(Y))
    return resultat

Theta_jouer = entrainer_multi(X_jouer, Y_jouer)

Theta_jouer

[{'ensoleile': {'_total_': 5, 'non': 3, 'oui': 2},
  'nuageux': {'_total_': 4, 'non': 0, 'oui': 4},
  'pluvieux': {'_total_': 5, 'non': 2, 'oui': 3}},
 {'chaude': {'_total_': 4, 'non': 2, 'oui': 2},
  'douce': {'_total_': 6, 'non': 2, 'oui': 4},
  'fraiche': {'_total_': 4, 'non': 1, 'oui': 3}},
 {'haute': {'_total_': 7, 'non': 4, 'oui': 3},
  'normale': {'_total_': 7, 'non': 1, 'oui': 6}},
 {'non': {'_total_': 8, 'non': 2, 'oui': 6},
  'oui': {'_total_': 6, 'non': 3, 'oui': 3}},
 {'non': 0.35714285714285715, 'oui': 0.6428571428571429}]

### I-4- Estimation de la probabilité de vraissemblance (loi multinomiale)
L'équation pour estimer la vraisemblance 
$$ P(f_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } f_j = v\}|}{|\{y = c_k\}|}$$

Si, dans le dataset de test, on veut calculer la probabilité d'une valeur $v$ qui n'existe pas dans le dataset d'entrainnement ou qui n'existe pas pour une classe donnée, on aura une probabilité nulle. Ici, on doit appliquer une fonction de lissage qui donne une petite probabilité aux données non vues dans l'entrainnement. Le lissage qu'on va utiliser est celui de Lidstone. Lorsque $\alpha = 1$ on l'appelle lissage de Laplace.
$$ P(f_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } f_j = v\}| + \alpha}{|\{y = c_k\}| + \alpha * |V|}$$
Où: 
- $\alpha$ est une valeur donnée 
- $V$ est l'ensemble des différentes valeurs de $f_j$ (le vocabulaire)

In [56]:
# TODO compléter cette fonction
def P_vraiss_multi(Theta_j, v, c, alpha=1.): 
    len_V = len(Theta_j) # La taille du vocabulaire
    nbr_c= nbr_v = 0
    for i in Theta_j.keys() :
        nbr_c += Theta_j[i][c]
        if i == v:
            nbr_v += Theta_j[i][c]
    return (nbr_v + alpha) / ( nbr_c + alpha * len_V )

# La probabilité de jouer si temps = pluvieux 
# P(temps = pluvieux | jouer=oui) = (nbr(temps=pluvieux et jouer=oui)+alpha)/(nbr(jour=oui) + alpha * nbr_diff(temps)))
# P(temps = pluvieux | jouer=oui) = (3 + 1)/(9 + 3) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = pluvieux | jouer=oui) = 4/12 ==> 0.33333333333333333333333333333333333~

# La probabilité de jouer si temps = neigeux 
# P(temps = neigeux | jouer=oui) = (nbr(temps=neigeux et jouer=oui)+alpha)/(nbr(jouer=oui) + alpha * nbr_diff(temps)))
# P(temps = neigeux | jouer=oui) = (0 + 1)/(9 + 3) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = neigeux | jouer=oui) = 1/13 ==> 0.0833333333333333333333333333333333333~


P_vraiss_multi(Theta_jouer_temps, "pluvieux", "oui"), P_vraiss_multi(Theta_jouer_temps, "neigeux", "oui")

(0.3333333333333333, 0.08333333333333333)

### I-5- Prédiction de la classe (loi multinomiale)
Revenons maintenant à notre équation de prédiction 
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$

Ici, vous devez prédire un seule échantillon $x$

In [73]:
# TODO compléter ce code
# Pour récupérer le théta de la caractéristique n°0 : Theta[0]
# anter est un booléen, si il est False, on ne compte pas la probabilité antérieure P(y = c_k)
def predire(x, Theta, alpha=1., anter=True): 
    c_opt = "" # la classe optimale
    p_c = Theta[-1] #les classes et leurs probabilités antérieures
    if not anter: # si on ne veut pas ajouter les probabiliés antérieures
        p_c = dict.fromkeys(p_c, 1.) # on définit le tous en 1; log(1) = 0
    max_log_p = np.NINF # - infinity 
   
    for index,key in enumerate(p_c.keys()):
        s = np.log(p_c[key])
        for i,j in enumerate(x):
            s += np.log(P_vraiss_multi(Theta[i],j,key,alpha))
        if s > max_log_p :
            max_log_p ,c_opt = s ,key 
        
    return c_opt, max_log_p

# Résultat: (('oui', -4.102643365036796), ('oui', -3.6608106127577567))
predire(["pluvieux", "fraiche", "normale", "oui"], Theta_jouer), predire(["pluvieux", "fraiche", "normale", "oui"], Theta_jouer, anter=False)


(('oui', -4.102643365036796), ('oui', -3.6608106127577567))

### I-7- Regrouper en une classe (loi multinomiale)

**Rien à programmer ici, il y a une petite analyse**


In [74]:
class NBMultinom(object): 
    
    def __init__(self, alpha=1.): 
        self.alpha = alpha
        
    def entrainer(self, X, Y):
        self.Theta = entrainer_multi(X, Y)
    
    def predire(self, X, anter=True, prob=False): 
        Y_pred = []
        for i in range(len(X)): 
            c, p = predire(X[i,:], self.Theta, alpha=self.alpha, anter=anter)
            if prob:
                Y_pred.append(p)
            else:
                Y_pred.append(c)
        return Y_pred

On va entrainer un modèle en utilisant notre imlémentation avec et sans probabilité antérieure. 
Normalement, on doit tester sur des données non vues (des données qu'on n'a pas utilisé pour l'entrainement). Mais, ici, on va tester sur les mêmes données d'entrainement afin de savoir si le modèle a bien représenté ce dataset ou non (calculer l'erreur) 

In [77]:
notre_modele = NBMultinom()
notre_modele.entrainer(X_jouer, Y_jouer)
Y_notre_ant = notre_modele.predire(X_jouer)
Y_notre_sans_ant = notre_modele.predire(X_jouer, anter=False)

# Ici, ce n'ai pas la peine d'exécuter plusieurs fois
# puisque le résultat sera le même 

# Le rapport de classification
from sklearn.metrics import classification_report

print("Notre modèle avec probabilité antérieure (a priori)")
print(classification_report(Y_notre_ant, Y_jouer))

print("Notre modèle sans probabilité antérieure (a priori)")
print(classification_report(Y_notre_sans_ant, Y_jouer))


Notre modèle avec probabilité antérieure (a priori)
              precision    recall  f1-score   support

         non       0.80      1.00      0.89         4
         oui       1.00      0.90      0.95        10

    accuracy                           0.93        14
   macro avg       0.90      0.95      0.92        14
weighted avg       0.94      0.93      0.93        14

Notre modèle sans probabilité antérieure (a priori)
              precision    recall  f1-score   support

         non       0.80      0.67      0.73         6
         oui       0.78      0.88      0.82         8

    accuracy                           0.79        14
   macro avg       0.79      0.77      0.78        14
weighted avg       0.79      0.79      0.78        14



**Analyser les résultats** 
<br>
On remarque que:
- Les deux modèles ont bien représenté le dataset
- Le modèle avec probabilité antérieure présente de meilleurs résultat que l'autre

## II- Détection de spam 

Ici, on va essayer d'appliquer l'apprentissage automatique sur la détection de spam. 
Chaque message dans le dataset est représenté en utilisant un modèle "Sac à mots" (BoW : Bag of Words).
Dans l'entrainement, on récupère les différents mots qui s'apparaissent dans les messages. 
Chaque mot va être considéré comme une caractéristique. 
Donc, pour chaque message, la valeur de la caractéristique est la fréquence de son mot dans le message. 
Par exemple, si le mot "good" apparait 3 fois dans le message, donc la caractéristique "good" aura la valeur 3 dans ce message.

Notre implémentation n'est pas adéquate pour la nature de ce problème. 
Dans Scikit-learn, le [sklearn.naive_bayes.CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html) est similaire à notre implémentation. 
L'algorithme adéquat pour ce type de problème est [sklearn.naive_bayes.MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

Le dataset utilisé est [SMS Spam Collection Dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset).
Les algorithmes comparés :
- Naive Bayes
- Arbre de décision
- Regression logistique 

### II-1- Préparation de données


In [84]:
messages = pd.read_csv("datasets/spam.csv", encoding="latin-1")
messages = messages.rename(columns={"v1": "classe", "v2": "texte"})
messages = messages.filter(["texte", "classe"])

messages.head()

,texte,classe
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


### II-2- Entrainement et test des modèles sur plusieurs exécutions 

Afin de satisfaire un étudiant qui réclame toujours sur le manque des données, nous avons décidé de comparer les algorithmes sur plusieurs excécutions (runs). 

**Rien à analyser ici**

**P.S.** timeit.default_timer() est dépendante du système d'exploitation. Aussi, elle peut être affectée par d'autre processus en parallèle. 

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import timeit
from sklearn.metrics import precision_score, recall_score

NBR_RUN = 7

temps_train = {
    "naive_bayes" : [],
    "arbre_decision": [],
    "reg_log": []
}

temps_test = {
    "naive_bayes" : [],
    "arbre_decision": [],
    "reg_log": []
}

perf = {
    "naive_bayes_P" : [],
    "arbre_decision_P": [],
    "reg_log_P": [], 
    "naive_bayes_R" : [],
    "arbre_decision_R": [],
    "reg_log_R": []
}


for run in range(NBR_RUN): 
    # prétaitement des données
    msg_train, msg_test, Y_train, Y_test = train_test_split(messages["texte"],messages["classe"],test_size=0.2)
    count_vectorizer = CountVectorizer()
    X_train = count_vectorizer.fit_transform(msg_train)
    X_test = count_vectorizer.transform(msg_test)
    
    # ==================================
    # ENTRAINEMENT 
    # ==================================
    
    #entrainement Naive Bayes
    naive_bayes = MultinomialNB()
    temps_debut = timeit.default_timer()
    naive_bayes.fit(X_train, Y_train)
    temps_train["naive_bayes"].append(timeit.default_timer() - temps_debut)
    
    #entrainement CART
    arbre_decision = DecisionTreeClassifier()
    temps_debut = timeit.default_timer()
    arbre_decision.fit(X_train, Y_train)
    temps_train["arbre_decision"].append(timeit.default_timer() - temps_debut)
    
    #entrainement Régression logitique
    reg_log = LogisticRegression(solver="lbfgs") #solver=sag est plus lent; donc j'ai choisi le plus rapide
    temps_debut = timeit.default_timer()
    reg_log.fit(X_train, Y_train)
    temps_train["reg_log"].append(timeit.default_timer() - temps_debut)
    
    # ==================================
    # TEST 
    # ==================================
    
    #test Naive Bayes
    temps_debut = timeit.default_timer()
    Y_naive_bayes = naive_bayes.predict(X_test)
    temps_test["naive_bayes"].append(timeit.default_timer() - temps_debut)
    
    
    #test CART
    temps_debut = timeit.default_timer()
    Y_arbre_decision = arbre_decision.predict(X_test)
    temps_test["arbre_decision"].append(timeit.default_timer() - temps_debut)
    
    #test Régression logitique
    temps_debut = timeit.default_timer()
    Y_reg_log = reg_log.predict(X_test)
    temps_test["reg_log"].append(timeit.default_timer() - temps_debut)
    
    # ==================================
    # PERFORMANCE 
    # ==================================
    # Ici, on va considérer une classification binaire avec une seule classe "spam" 
    # On ne juge pas le classifieur sur sa capacité de détecter les non spams
    
    perf["naive_bayes_P"].append(precision_score(Y_test, Y_naive_bayes, pos_label="spam"))
    perf["arbre_decision_P"].append(precision_score(Y_test, Y_arbre_decision, pos_label="spam"))
    perf["reg_log_P"].append(precision_score(Y_test, Y_reg_log, pos_label="spam"))
    
    perf["naive_bayes_R"].append(recall_score(Y_test, Y_naive_bayes, pos_label="spam"))
    perf["arbre_decision_R"].append(recall_score(Y_test, Y_arbre_decision, pos_label="spam"))
    perf["reg_log_R"].append(recall_score(Y_test, Y_reg_log, pos_label="spam"))
    
    

temps_train

{'naive_bayes': [0.007559700001365854,
  0.007837999999537715,
  0.007951999999932013,
  0.008776999999099644,
  0.013415100000202074,
  0.015027999999801978,
  0.008826700001009158],
 'arbre_decision': [0.09238189999996393,
  0.09128590000000258,
  0.105362199999945,
  0.11421980000159238,
  0.10148550000121759,
  0.10758549999991374,
  0.11417330000040238],
 'reg_log': [0.05187370000021474,
  0.0499601999999868,
  0.048833900000317954,
  0.08134880000034173,
  0.05943159999878844,
  0.04652689999966242,
  0.07069619999856513]}

### II-3- Analyse du temps d'apprentissage 

Combien de temps chaque algorithme prend pour entrainer le même dataset d'entrainement


In [86]:
pd.DataFrame(temps_train)

,naive_bayes,arbre_decision,reg_log
0,0.007560,0.092382,0.051874
1,0.007838,0.091286,0.049960
2,0.007952,0.105362,0.048834
3,0.008777,0.114220,0.081349
4,0.013415,0.101486,0.059432
5,0.015028,0.107585,0.046527
6,0.008827,0.114173,0.070696


**Analyser**
En comparant les temps d'apprentissage de chaque algorithme sur ce dataset on constate que :
<br>
- Le classifieur Naive bayse présente les meilleurs temps d'entrainement, ceci confirme le fait que cet algorithme est adapté pour le filtrage du courrier indésirable et de classification de documents.
<br>
- La régression logistique aussi présente de bons temps d'entrainement dans ce cas là, contrairement aux arbres de décision qui prennent un temps relativement plus élévé, et celà est du au nombre important de caractéristique (features), vu que chaque mot représente une.

### II-4- Analyse du temps de test 

Combien de temps chaque algorithme prend pour prédir les classes

In [89]:
pd.DataFrame(temps_test)

,naive_bayes,arbre_decision,reg_log
0,0.000196,0.000357,0.000089
1,0.000227,0.000357,0.000090
2,0.000402,0.000648,0.000192
3,0.000376,0.000817,0.000183
4,0.000335,0.000505,0.000139
5,0.000195,0.000357,0.000088
6,0.000191,0.000363,0.000092


**Analyser**
<br>
Maintenant en comparant les temps de prédiction de chaque algorithme:
- la régression logistique présente les meilleurs résultat, vu que la prédiction pour cet algorithme, ne s'agit que de calculer une valeur en utilisant le vecteur des theta et la comparer avec un seuil pour pouvoir la classifier ( Ici il s'agit d'une regression logistique binaire spam / non spam )
- La classifieur naive bayes vient en deuxième position, car pour prédire il faut calculer la vraissemblance de chacune des caractéristiques pour chaque valeur de y, et l'antérieure des valeurs des y.
- Les arbre de décision, dans ce cas l'algorithme CART vient en dernier pour ce dataset, car pour prédire il faut parcourir l'arbre jusqu'à arriver à une feuille, et vu le nombre des caractéristique (Chaque mot représente une), l'arbre est trop profond.

### II-5- Analyse de la performance 

Ici, on compare les modèles en se basant sur leurs capacités à détecter le spam. 
On va utiliser la précision et le rappel.

In [93]:
df = pd.DataFrame(perf, columns = ["arbre_decision_P", "naive_bayes_P", "reg_log_P", "arbre_decision_R", "naive_bayes_R", "reg_log_R"])
df

,arbre_decision_P,naive_bayes_P,reg_log_P,arbre_decision_R,naive_bayes_R,reg_log_R
0,0.894737,0.978102,0.976378,0.843972,0.950355,0.879433
1,0.885906,0.980000,0.986014,0.851613,0.948387,0.909677
2,0.937931,0.979452,0.992857,0.860759,0.905063,0.879747
3,0.872483,0.985714,0.984733,0.860927,0.913907,0.854305
4,0.915385,0.968504,0.967213,0.875000,0.904412,0.867647
5,0.926174,0.986755,0.979021,0.890323,0.961290,0.903226
6,0.869565,0.984252,0.975806,0.882353,0.919118,0.889706


In [94]:
df.mean()

arbre_decision_P    0.900312
naive_bayes_P       0.980397
reg_log_P           0.980289
arbre_decision_R    0.866421
naive_bayes_R       0.928933
reg_log_R           0.883391
dtype: float64

**Analyser**

- Pour la précision, Le naive bayes et la regression logistique présente des résultats presque similaires et qui sont mieux que ceux présenté par les arbres de décision
- Pour le recall, c'est le naive bayes qui a les meilleurs résultats, et puis vient la regression logistique et en dernier les arbres de décision.
- Comme conclusion, on peut dire que les arbres de décision ne sont pas de bons classifieur lorsque il s'agit du text. Sinon entre la regression logistique et le naive bayes, si on préfère un meilleur recall, on doit choisir le naive bayes, et si on prefère une prédiction plus rapide on doit choisir la regression logistique.